# Regression Exercises

## 1. Acquire bedroomcnt, bathroomcnt, calculatedfinishedsquarefeet, taxvaluedollarcnt, yearbuilt, taxamount, and fips from the zillow database for all 'Single Family Residential' properties.

In [165]:
# Imports
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import env
import os

In [166]:
# Read data from the student_grades table in the school_sample database on our mySQL server. 

def get_connection(db, user=env.user, host=env.host, password=env.password):
    return f'mysql+pymysql://{user}:{password}@{host}/{db}'

def get_zillow_data():
    filename = "sfr_2017.csv"

    if os.path.isfile(filename):

        return pd.read_csv(filename, index_col=0)
    else:
        # Create the url
        url = get_connection('zillow')

        # Read the SQL query into a dataframe
        df = pd.read_sql('''SELECT 
            bathroomcnt, 
            bedroomcnt, 
            calculatedfinishedsquarefeet,
            taxvaluedollarcnt,
            yearbuilt,
            taxamount,
            fips
        FROM properties_2017
        JOIN propertylandusetype
            ON propertylandusetype.propertylandusetypeid = properties_2017.propertylandusetypeid
        WHERE propertylandusetype.propertylandusetypeid = '261';'''
        , url)


        # Write that dataframe to disk for later. Called "caching" the data for later.
        df.to_csv(filename)

        # Return the dataframe to the calling code
        return df


In [167]:
df = get_zillow_data()

In [168]:
df.columns

Index(['bathroomcnt', 'bedroomcnt', 'calculatedfinishedsquarefeet',
       'taxvaluedollarcnt', 'yearbuilt', 'taxamount', 'fips'],
      dtype='object')

In [169]:
df.rename(columns={'bathroomcnt' : 'bath', 'bedroomcnt' : 'bed', 'calculatedfinishedsquarefeet' : 'sqft', 'taxvaluedollarcnt' : 'tax_val', 'yearbuilt' : 'year', 'taxamount' : 'tax'}, inplace=True)

## 2. Using your acquired Zillow data, walk through the summarization and cleaning steps in your wrangle.ipynb file like we did above. You may handle the missing values however you feel is appropriate and meaningful; remember to document your process and decisions using markdown and code commenting where helpful.


In [170]:
df.head()

,bath,bed,sqft,tax_val,year,tax,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
3,0.0,0.0,NaN,2108.0,NaN,174.21,6037.0
4,2.0,4.0,3633.0,296425.0,2005.0,6941.39,6037.0


In [171]:
# 2152863 rows and 7 columns
df.shape

(2152863, 7)

In [172]:
# Display readable summary statistics for numeric columns. All 7 columns are here.
df.describe().T

,count,mean,std,min,25%,50%,75%,max
bath,2152852.0,2.230688,0.999280,0.00,2.00,2.00,3.00,32.00
bed,2152852.0,3.287196,0.954754,0.00,3.00,3.00,4.00,25.00
sqft,2144379.0,1862.855178,1222.125124,1.00,1257.00,1623.00,2208.00,952576.00
tax_val,2152370.0,461896.237963,699676.049600,1.00,188170.25,327671.00,534527.00,98428909.00
year,2143526.0,1960.949681,22.162196,1801.00,1949.00,1958.00,1976.00,2016.00
tax,2148421.0,5634.865978,8178.910249,1.85,2534.98,4108.95,6414.32,1337755.86
fips,2152863.0,6048.377335,20.433292,6037.00,6037.00,6037.00,6059.00,6111.00


In [173]:
# All columns are float
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2152863 entries, 0 to 2152862
Data columns (total 7 columns):
 #   Column   Dtype  
---  ------   -----  
 0   bath     float64
 1   bed      float64
 2   sqft     float64
 3   tax_val  float64
 4   year     float64
 5   tax      float64
 6   fips     float64
dtypes: float64(7)
memory usage: 115.0 MB


In [174]:
# Check for nulls
df.isnull().sum()

bath         11
bed          11
sqft       8484
tax_val     493
year       9337
tax        4442
fips          0
dtype: int64

In [175]:
# Check for any 0 values
print(df[df.bath==0].shape)
print(df[df.bed==0].shape)
print(df[df.sqft==0].shape)
print(df[df.tax_val==0].shape)
print(df[df.year==0].shape)
print(df[df.tax==0].shape)


(13027, 7)
(13187, 7)
(0, 7)
(0, 7)
(0, 7)
(0, 7)


In [176]:
df[df.bed==0][df.bath==0].shape

(12088, 7)

In [177]:
df[df.bed==0][df.bath==0].sqft.isna().sum()

8255

In [178]:
df[df.bed==0][df.bath==0].sqft.isna().sum()

8255

In [179]:
df[df.bed==0][df.bath==0][df.year.isna()].sqft.isna().sum()

7735

In [180]:
df[df.bed==0][df.bath==0][df.year.isna()][df.tax.isna()].sqft.isna().sum()

1709

In [181]:
df[df.tax.isna()][df.bed==0]

,bath,bed,sqft,tax_val,year,tax,fips
0,0.0,0.0,NaN,27516.0,NaN,NaN,6037.0
1,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
2,0.0,0.0,NaN,10.0,NaN,NaN,6037.0
5,0.0,0.0,NaN,124.0,NaN,NaN,6037.0
10,0.0,0.0,NaN,11009.0,NaN,NaN,6037.0
...,...,...,...,...,...,...,...
2147119,0.0,0.0,NaN,5758.0,NaN,NaN,6037.0
2147176,0.0,0.0,NaN,13060.0,NaN,NaN,6037.0
2147189,0.0,0.0,936.0,32585.0,1913.0,NaN,6037.0
2149353,0.0,0.0,324.0,15659.0,1946.0,NaN,6037.0


<div class="alert alert-block alert-success">

#### <b>Summarize Takeaways</b>

- 2152863 rows and 7 columns
- All columns are float
- Nulls:
    - bath                       11
    - bed                        11
    - sqft                       8484 (of these there are 8255 that have no info for beds or baths)
    - tax_value                  493
    - year                       9337
    - tax                        4442 (of these there are 1709 that have no info for bed, bath, year)
- Zeros:
    - bath 13027
    - bed 13187
    - There are 12088 that have a zero value for both beds and baths.
    - There are 8255 that have a zero value for bot beds and baths AND sqft is null
    - There are 7735 that have a zero value for bot beds and baths AND sqft is null AND year is null
    

<b>Will drop all nulls and zeros. Most observations with any nulls or zeros are missing data in multiple columns. Also the missing data makes up less than a percent of total.</b>

In [182]:
df.replace(0, np.nan, inplace=True)
df = df.dropna()

In [183]:
df.isnull().sum()

bath       0
bed        0
sqft       0
tax_val    0
year       0
tax        0
fips       0
dtype: int64

## 3. Store all of the necessary functions to automate your process from acquiring the data to returning a cleaned dataframe with no missing values in your wrangle.py file. Name your final function wrangle_zillow.


In [184]:
def wrangle_zillow():
    def get_connection(db, user=env.user, host=env.host, password=env.password):
        return f'mysql+pymysql://{user}:{password}@{host}/{db}'

    def get_zillow_data():
        filename = "sfr_2017.csv"

        if os.path.isfile(filename):

            return pd.read_csv(filename, index_col=0)
        else:
            # Create the url
            url = get_connection('zillow')

            # Read the SQL query into a dataframe
            df = pd.read_sql('''SELECT 
                bathroomcnt, 
                bedroomcnt, 
                calculatedfinishedsquarefeet,
                taxvaluedollarcnt,
                yearbuilt,
                taxamount,
                fips
            FROM properties_2017
            JOIN propertylandusetype
                ON propertylandusetype.propertylandusetypeid = properties_2017.propertylandusetypeid
            WHERE propertylandusetype.propertylandusetypeid = '261';'''
            , url)


            # Write that dataframe to disk for later. Called "caching" the data for later.
            df.to_csv(filename)

            # Return the dataframe to the calling code
            return df
    df = get_zillow_data()
    df.replace(0, np.nan, inplace=True)
    df = df.dropna() 
    return df   

In [185]:
wrangle_zillow()

,bathroomcnt,bedroomcnt,calculatedfinishedsquarefeet,taxvaluedollarcnt,yearbuilt,taxamount,fips
4,2.0,4.0,3633.0,296425.0,2005.0,6941.39,6037.0
6,4.0,3.0,1620.0,847770.0,2011.0,10244.94,6037.0
7,2.0,3.0,2077.0,646760.0,1926.0,7924.68,6037.0
18,1.0,3.0,1244.0,169471.0,1950.0,2532.88,6037.0
19,2.0,3.0,1300.0,233266.0,1950.0,3110.99,6037.0
...,...,...,...,...,...,...,...
2152856,4.0,4.0,4375.0,422400.0,2015.0,13877.56,6037.0
2152858,3.0,4.0,2262.0,960756.0,2015.0,13494.52,6059.0
2152859,4.5,4.0,3127.0,536061.0,2014.0,6244.16,6059.0
2152861,2.5,3.0,1974.0,424353.0,2015.0,5302.70,6059.0
